<a href="https://colab.research.google.com/github/liviator/NoSQLProject/blob/master/CLassification_journaux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from requests import get
import bs4 as bs
import urllib.request
import nltk
from bs4 import BeautifulSoup
import pandas as pd
import string
import numpy as np
import matplotlib
from matplotlib import pyplot as pyp
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk import RegexpTokenizer
from heapq import nlargest
#from textblob import TextBlob
#from textblob_fr import PatternTagger, PatternAnalyzer    
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def Check_Web_Connection(actual_url):
    response = get(actual_url)
    return response

def Get_Web_Soup(response):
    html_soup = BeautifulSoup(response.text,"html.parser")#Récupère l'ensemble du site web 
    return html_soup

def Get_URL_main_articles(soup):#Récupération des articles mis en avant par le site
    body = soup.find(id='main-content')
    temporary_url_array = []
    main_art = body.find_all('div',class_="dm-block dm-block-bloc_1_news")
    for elem in main_art:
        url = elem.find('a',href=True)
        temporary_url_array.append(url['href'])
    url_array = [temporary_url_array[0],temporary_url_array[1],temporary_url_array[2],temporary_url_array[3]]
    #Sur CNews, les derniers gros titres correspondent souvent à de la culture et non à des informations sur l'actualité
    return(url_array)

def Get_URL_sub_articles(soup):#Récupération des autres articles
    body = soup.find(id='main-content')
    temporary_url_array = []
    url_array = []
    main_art = body.find_all('div',class_="dm-block dm-block-bloc_3_news")
    for elem in main_art:
        url = elem.find_all('a',href=True)
        for i in url:
            temporary_url_array.append(i['href'])
    for i in range(15): #Ce nombre d'URL peut être modifié en fonction du nombre d'article qu'on veut résumer
        url_array.append(temporary_url_array[i])
    return(url_array)

def Get_text_from_article(actual):#Récupère l'ensemble des morceaux de texte contenu dans l'article, contient également les textes assosiés aux tweets cités ainsi que les URL que je n'ai pas réussi à retirer 
    actual_url= main_url + actual[1:]
    text = []
    response = Check_Web_Connection(actual_url)
    soup = Get_Web_Soup(response)#On récupère l'ensemble du site avant d'y appliquer des recherches via soup.find
    body= soup.find(id="wrapper-publicite")
    titre = body.find("h1",class_="article-title")
    if titre != None: #Il y a quelques problèmes avec l'uniformité des pages du site web, le titre est récupéré sur certains articles mais pas sur tous
        text.append(titre.text + ".")
        text.append(" ")
    intro = body.find("p", class_="dm_article-chapeau")
    text.append(intro.text)
    text.append(" ")
    content_1 = body.find_all("p",class_="dm_article-paragraph")
    content_2 = body.find_all('p',class_= None)#Certains morceaux de texte ne sont associés à aucune classe
    for elem in content_1:
        text.append(elem.text)
        text.append(' ')
    for elem in content_2:
        text.append(elem.text)
        text.append(' ')
    article = ''.join(text)
    return article

def Print_all_articles():#Affiche tout les articles du site
    #ON commence par afficher les grands articles du site
    response = Check_Web_Connection(main_url)
    soup = Get_Web_Soup(response)
    url_prince = Get_URL_main_articles(soup)
    url_sub = Get_URL_sub_articles(soup)
    print("Voici les articles correspondant aux gros titres de Cnews\n")
    for elem in url_prince:  
        print(Get_text_from_article(elem))
        print("\n"*3)
    print("Voici tout les autres articles proposé par Cnews")
    for elem in url_sub:  
        print(Get_text_from_article(elem))
        print("\n"*3)
        
        
def Get_num(article):#Obtiens un dictionnaire indiquant le nombre d'ittération de chaque mot de l'article
    num = {}
    tokenizer = RegexpTokenizer(r'''\w'|\w+|[^\w\s]''') #J'utilise RegexpTokenizer pour les mots car en utilisant word_tokenize, j'avais certains problèmes causés par la langue française / les textes dans les tweets
    tok_art = tokenizer.tokenize(article)
    for word in tok_art:
        if word.lower() not in sw:
            if word.lower() not in punctuation: 
                if word in num.keys():
                    num[word] = num[word] + 1 #On compte le nombre de fois qu'un mot apparait dans le texte 
                else:
                    num[word] = 1
    return num


def Get_value_sentence(sentence, num):#Retourne le poids d'une phrase de l'article
    tokenizer = RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')
    sent_tok = tokenizer.tokenize(sentence)
    count = 0
    for word in sent_tok:#On calcule le poids de chaque mots de la phrase grâce au dictionnaire de fréquence qu'on à déterminé précédemment
        if word.lower() not in sw:
            if word.lower() not in punctuation:
                count = count + num[word]
    count = count / len(sent_tok)#On effectue une moyenne du poids de chaque mots de façon à ne pas avantager les phrases très longues qui peuvent s'avérer ne pas être essentielles
    return count

def Get_weight_sentences(num,article):#Retourne le poids de chaque phrase de l'article
    tok_art = sent_tokenize(article, language='french')#on tokenize l'articles en phrases de façon à pouvoir évaluer le poids de chaque phrase
    weight = {}    
    for sentences in tok_art:
        weight[sentences] = Get_value_sentence(sentences,num)
    max_val = weight.values()
    weight[tok_art[0]] = max(max_val)#Permet d'avoir forcement le titre de l'article dans son résumé ce qui rend la compréhension de l'article plus simple
    return weight  

def Summary_article(article):
    num = Get_num(article)
    weight = Get_weight_sentences(num,article)
    length = int(len(weight)*0.25)#Ce nombre peut être modifié selon la taille de résumé que l'on désire, actuellement, on récupère les 25% des phrases de l'articles qui ont le plus grand poids
    summary_array = nlargest(length,weight,key=weight.get)#Renvoie un tableau contenant un pourcentage des phrases ayant le plus grand poids dans l'article
    summary= ''.join(summary_array)
    return summary

def Sentiment_article(article): #Permet d'obtenir le sentiment dégagé d'un article: 0 étant plutot neutre, 1 positif et -1 négatif
    tok_art = sent_tokenize(article, language='french')
    sentiment = 0
    for sentences in tok_art:
        sentiment = sentiment + TextBlob(sentences,pos_tagger = PatternTagger(), analyzer = PatternAnalyzer()).sentiment[0]
    sentiment = sentiment / len(tok_art)
    return sentiment

def Print_sentiment(sentiment_array): #Permet d'afficher la polarité de tout les articles
    pyp.plot(sentiment_array)
    axes = pyp.gca()
    axes.set_ylim(-1,1)
    pyp.xlabel("Positionnement de l'article dans le fil d'actualité")
    pyp.ylabel("Polarité de l'article (1 = positif, -1 négatif , 0 neutre)")
    pyp.suptitle("Polarité des articles actuels de CNEWS")

def Print_detail_article():
    reponse = Check_Web_Connection(main_url)
    main_soup = Get_Web_Soup(reponse)
    text = Get_text_from_article(test_url)
    print("L'article \n")
    print(text + "\n"*4)
    num = Get_num(text)
    sorted_num = {}
    sorted_keys = sorted(num,key=num.get, reverse = True)
    for elem in sorted_keys:
        sorted_num[elem]=num[elem]
    num_list = sorted_num.items()
    temp_x,temp_y = zip(*num_list)
    x=temp_x[:7]
    y= temp_y[:7]
    pyp.plot(x,y)
    pyp.ylabel("Nombre d'ittération")
    pyp.xlabel("Mots les plus employés dans l'article")
    pyp.suptitle("Répartition des mots dans l'article")
    print("\n"*4)
    print("Le poids de chaque phrase de l'article")
    print(Get_weight_sentences(num,text))
    print("\n"*4)
    print("Le résumé de l'article : " + Summary_article(text))
    print("\n"*4 + "La polarité de l'article (-1 négatif, 0 neutre, 1 positif)")
    print(Sentiment_article(text))
    
def Synthese_sentiment():
    reponse = Check_Web_Connection(main_url)
    main_soup = Get_Web_Soup(reponse)
    url_princ = Get_URL_main_articles(main_soup)
    url_sub = Get_URL_sub_articles(main_soup)
    sent_array = []
    for articles in url_princ:
        text = Get_text_from_article(articles)
        sent_array.append(Sentiment_article(text))
    
    for articles in url_sub:
        text = Get_text_from_article(articles)
        sent_array.append(Sentiment_article(text))
    Print_sentiment(sent_array)
    
def Resume_articles():
    reponse = Check_Web_Connection(main_url)
    main_soup = Get_Web_Soup(reponse)
    url_princ = Get_URL_main_articles(main_soup)
    url_sub = Get_URL_sub_articles(main_soup)
    sent_array = []
    for articles in url_princ:
        text = Get_text_from_article(articles)
        print(Summary_article(text))
        print("\n")
    
    
    for articles in url_sub:
        text = Get_text_from_article(articles)
        print(Summary_article(text))
        print("\n")

def Fetch_Url(url, array):
    tempo_arr = []
    response = Check_Web_Connection(url)
    soup = Get_Web_Soup(response)
    body = soup.find(id='app')
    articles = body.find_all('div',class_="sc-AxjAm sc-1ttlxdz-0 bTKqUl")
    for elem in articles:
        url = elem.find_all('a',href=True)
        for i in url:
            tempo_arr.append(i['href'])
    for i in range(50): # defini le nbr d'article prix
        
        if i%2 == 0:
            array.append(tempo_arr[i])


In [ ]:
urls

['/tech-medias/medias/la-formule-1-a-reconquis-les-telespectateurs-francais-1301585',
 '/tech-medias/hightech/tencent-se-prepare-au-tour-de-vis-de-pekin-apres-une-excellente-annee-2020-1301537',
 '/tech-medias/hightech/leurope-doit-investir-300-milliards-deuros-dans-ses-reseaux-telecoms-selon-les-operateurs-1301470',
 '/tech-medias/hightech/puces-une-penurie-qui-menace-les-pme-de-tous-les-secteurs-de-leconomie-1301259',
 '/tech-medias/medias/netflix-ne-veut-pas-seulement-financer-lexception-culturelle-mais-aussi-en-beneficier-1301258',
 '/tech-medias/hightech/pourquoi-intel-entre-en-co-opetition-avec-tsmc-1301232',
 '/tech-medias/hightech/reseau-cuivre-dorange-larcep-denonce-un-calvaire-dans-les-campagnes-1301225',
 '/tech-medias/hightech/en-plein-boom-la-messagerie-chiffree-telegram-leve-1-milliard-de-dollars-de-dettes-1301130',
 '/tech-medias/hightech/amazon-accelere-dans-la-production-de-jeux-video-et-recrute-chez-ubisoft-1301108',
 '/weekend/business-story/semi-conducteurs-tsmc-ce-

In [ ]:
main_url = 'https://www.lesechos.fr/tech-medias/hightech/reseau-cuivre-dorange-larcep-denonce-un-calvaire-dans-les-campagnes-1301225'
response = get(main_url)
html_soup = BeautifulSoup(response.text,'html.parser')

In [ ]:
url_array = []
pol = 'https://www.lesechos.fr/politique-societe'
eco="https://www.lesechos.fr/economie-france"
monde="https://www.lesechos.fr/monde"
tech="https://www.lesechos.fr/tech-medias"
entreprise="https://www.lesechos.fr/industrie-services"
bourse="https://www.lesechos.fr/bourse"
finance="https://www.lesechos.fr/finance-marches"
region="https://www.lesechos.fr/pme-regions"
patrimoine="https://www.lesechos.fr/patrimoine"
url_array.append(Fetch_Url(pol,url_array))
url_array.append(Fetch_Url(eco,url_array))
url_array.append(Fetch_Url(monde,url_array))
url_array.append(Fetch_Url(tech,url_array))
url_array.append(Fetch_Url(entreprise,url_array))
url_array.append(Fetch_Url(bourse,url_array))
url_array.append(Fetch_Url(finance,url_array))
url_array.append(Fetch_Url(region,url_array))
url_array.append(Fetch_Url(patrimoine,url_array))

print(url_array)

['/politique-societe/politique/islamisme-la-majorite-a-loffensive-contre-les-ecologistes-1301625', '/politique-societe/societe/ile-de-france-lidee-davancer-les-vacances-scolaires-fait-debat-1301612', '/politique-societe/politique/sondage-exclusif-elections-regionales-en-marche-face-au-risque-du-toboggan-1301557', '/politique-societe/societe/covid-la-france-joue-sur-la-crise-pour-attirer-les-etudiants-etrangers-1301499', '/politique-societe/societe/loi-climat-les-commercants-defendent-les-ecrans-numeriques-dans-leurs-vitrines-1301466', '/politique-societe/politique/regionales-dans-les-hauts-de-france-la-partie-delicate-de-xavier-bertrand-face-a-une-gauche-unie-et-un-rn-tres-implante-1301448', '/politique-societe/emmanuel-macron-president/macron-comment-sauver-le-marqueur-europeen-1301373', '/politique-societe/gouvernement/covid-lunion-europeenne-a-manque-dambition-sur-les-vaccins-estime-macron-1301302', '/politique-societe/emmanuel-macron-president/covid-la-confiance-envers-lexecutif-au

In [ ]:
len(url_array)

234

In [ ]:
content = []
new_url = []
label = []
titles = []
for i in url_array: 
  url = 'https://www.lesechos.fr'
  if i !=None and '.php'not in i:
    url+= str(i) #in order to get the differents pages url
    print(url)
    response = get(url)
    html_soup = BeautifulSoup(response.text,'html.parser')
    body = html_soup.find(id='app')
    main_art = body.find_all('p',class_="sc-AxirZ bUHJAM")
    title = body.find('header',class_='cyue7g-0 bWrNQA')
    titles.append(title.get_text())
    print(title.get_text())
    fullarticle = ''
    for itm in main_art:
      fullarticle += itm.get_text().replace('\xa0',' ').replace('\u202f',' ')
    content.append(''.join([i for i in fullarticle if not i.isdigit()]))
    new_url.append(url+str(i))
    label.append(str(i).split('/')[1])

https://www.lesechos.fr/politique-societe/politique/islamisme-la-majorite-a-loffensive-contre-les-ecologistes-1301625
Islamisme : la majorité à l'offensive contre les écologistesProfitant d'une subvention contestée pour la construction d'une mosquée à Strasbourg, plusieurs ministres sont montés au créneau pour dénoncer ce qu'il considère comme une forme de complaisance d'Europe Ecologie-Les Verts à l'égard de l'islamisme radical. Le parti écologiste dénonce une « diabolisation ».Politique
https://www.lesechos.fr/politique-societe/societe/ile-de-france-lidee-davancer-les-vacances-scolaires-fait-debat-1301612
Ile-de-France : l'idée d'avancer les vacances scolaires fait débatLa proposition de la présidente de la région Ile-de-France d'avancer de deux semaines les vacances scolaires de printemps est accueillie tièdement au ministère de l'Education nationale. Le principal syndicat d'enseignants du primaire, le SNUipp-FSU, va appeler à la grève pour dénoncer la politique du « déni » du gouve

In [ ]:
data  = pd.DataFrame({
        'titles': titles,
        'contents': content,
        'urls': new_url,
        'labels': label
})

In [ ]:
len(content)

200

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from nltk.corpus import stopwords



In [ ]:
nltk.download("stopwords")
nltk.download('punkt')


sw = stopwords.words('french') 
sw.extend(['a',"d'","n'","l'",'plus','moins','être','comme','avoir','ici','faire','faut','non','dit','ans','2020','2021','cette','mois','depuis','tous','contre','sous','cet'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

vectorizer = TfidfVectorizer(stop_words=sw)
text = data['contents'].values.astype("U")
X = vectorizer.fit_transform(text)

In [ ]:
true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [ ]:
result =[]

for i in range(true_k):
  print("Cluster %d:" + str(i))
  word_number = 10
  temp = []
  for ind in order_centroids[i, :word_number]:
    temp.append(terms[ind])
    
    print(terms[ind])
  result.append(temp)

data['cluster']= model.labels_

Cluster %d:0
prix
france
banque
an
hausse
selon
crise
français
entreprises
baisse
Cluster %d:1
covid
réanimation
lits
vaccination
patients
france
soins
établissements
vont
ile
Cluster %d:2
vaccination
doses
vaccins
vaccin
astrazeneca
covid
millions
ue
pays
européenne
Cluster %d:3
travail
france
gouvernement
entreprise
sociaux
salariés
entreprises
après
fait
covid
Cluster %d:4
projet
ville
bio
espaces
association
euros
commerce
lorient
maires
texte
Cluster %d:5
usines
puces
tsmc
électroniques
intel
géant
semi
production
composants
philips
Cluster %d:6
millions
production
eau
euros
site
groupe
salaires
deux
directeur
six
Cluster %d:7
dette
finances
dépenses
rembourser
arthuis
loyer
bailleurs
sociale
rapport
publiques
Cluster %d:8
milliards
euros
épargne
assurance
abattement
vie
solidaire
télécoms
plafond
impôt
Cluster %d:9
régionales
élections
pea
bertrand
xavier
présidentielle
scrutin
port
tour
campagne


In [ ]:
print(“\n”)
print(“Prediction”)
X = vectorizer.transform(["Nothing is easy in cricket. Maybe when you watch it on TV, it looks easy. But it is not. You have to use your brain and time the ball."])
predicted = model.predict(X)
print(predicted)

In [ ]:
pd.set_option('display.max_rows', None)
data.head()

,titles,contents,urls,labels,cluster
0,Islamisme : la majorité à l'offensive contre l...,L'occasion était trop belle pour ne pas la sai...,https://www.lesechos.fr/politique-societe/poli...,politique-societe,4
1,Ile-de-France : l'idée d'avancer les vacances ...,"A une semaine de l'échéance potentielle, la qu...",https://www.lesechos.fr/politique-societe/soci...,politique-societe,1
2,ExclusifSONDAGE EXCLUSIF - Elections régionale...,"Ce n'est encore qu'un tassement, mais c'est dé...",https://www.lesechos.fr/politique-societe/poli...,politique-societe,9
3,ExclusifCovid : la France joue sur la crise po...,L'embellie avec la pandémie. « En termes d'att...,https://www.lesechos.fr/politique-societe/soci...,politique-societe,0
4,Loi Climat : les commerçants défendent les écr...,A quelques jours du début de son examen par le...,https://www.lesechos.fr/politique-societe/soci...,politique-societe,4


In [ ]:
def createComparisonArray(labels): # function that takes an iterable and returns an array with true value for each value wich are the same base on the index
  label_array = []
  for i in  labels:
    temp =[]
    for j in labels:
      if i == j:
        temp.append(True)
      else:
        temp.append(False)
    label_array.append(temp)
  return label_array

In [ ]:
labels_comparison = createComparisonArray(data['labels'])
cluster_array = createComparisonArray(data['cluster'])

In [ ]:
def array_comparison(first_array,second_array): #function to compare to array and 
  res=0
  for i in range(0,len(first_array)):
    for j in range(0,len(first_array[0])):
      if first_array[i][j]== second_array[i][j]:
        res+=1
  final_res = res / (len(first_array)*len(first_array[0]))
  
  return final_res



In [ ]:
a=array_comparison(label_array,cluster_array)
a

40000


0.7887

In [69]:
pd.set_option('display.max_rows', None)
def title_associated(target, data):

  count = 0
  for eleme in data['cluster']:
    if eleme == target:
      print(data['titles'][count])
      print(eleme)
    count = count+1


ExclusifLustucru prêt à racheter Panzani pour créer un champion français des pâtesL'Espagnol Ebro, propriétaire des pâtes Panzani, se dit ouvert à une vente s'il obtient environ 600 millions d'euros. Le groupe Lustucru, associé avec le fonds d'investissement CVC est sur les rangs.Conso & Distribution
2
Santander réorganise son réseau au Royaume-Uni et supprime 600 emploisLa filiale britannique de la banque espagnole a annoncé la fermeture de quatre sites ainsi que le déplacement de son siège de Londres vers Milton Keynes. La pandémie ayant accéléré la transition vers les services de banque en ligne, elle compte également fermer 111 agences, soit 20 % de son réseau, ce qui devrait entraîner quelque 600 suppressions d'emplois.Banque & Assurances
2
Cinq clés pour comprendre les avantages et inconvénients d'un SPACPour les start-up, ce mécanisme permet de se soustraire au long processus d'introduction en Bourse classique. Il offre aussi généralement un prix d'introduction plus élevé.Marché